# Classes

## Resumo

Classes proporcionam uma forma de organizar dados e funcionalidades juntos. Criar uma nova classe cria um novo `tipo` de objeto, permitindo que novas `instâncias` desse tipo sejam produzidas. Cada instância da classe pode ter atributos anexados a ela, para manter seu estado. Instâncias da classe também podem ter métodos (definidos pela classe) para modificar seu estado.

Em comparação com outras linguagens de programação, o mecanismo de classes de Python introduz a programação orientada a objetos sem acrescentar muitas novidades de sintaxe ou semântica. É uma mistura de mecanismos equivalentes encontrados em C++ e Modula-3. As classes em Python oferecem todas as características tradicionais da programação orientada a objetos: o mecanismo de herança permite múltiplas classes base (herança múltipla), uma classe derivada pode sobrescrever quaisquer métodos de uma classe ancestral e um método pode invocar outro método homônimo de uma classe ancestral. Objetos podem armazenar uma quantidade arbitrária de dados de qualquer tipo. Assim como acontece com os módulos, as classes fazem parte da natureza dinâmica de Python: são criadas em tempo de execução, e podem ser alteradas após sua criação.

Usando a terminologia de C++, todos os membros de uma classe (incluindo dados) são públicos (veja exceção abaixo Variáveis privadas), e todos as funções membro são virtuais. Como em Modula-3, não existem atalhos para referenciar membros do objeto de dentro dos seus métodos: o método (função definida em uma classe) é declarado com um primeiro argumento explícito representando o objeto (instância da classe), que é fornecido implicitamente pela chamada ao método. Como em Smalltalk, classes são objetos. Isso fornece uma semântica para importar e renomear. Ao contrário de C++ ou Modula-3, tipos pré-definidos podem ser utilizados como classes base para extensões por herança pelo usuário. Também, como em C++, a maioria dos operadores (aritméticos, indexação, etc) podem ser redefinidos por instâncias de classe.

## 1. Uma palavra sobre nomes e objetos

Objetos têm individualidade, e vários nomes (em diferentes escopos) podem ser vinculados a um mesmo objeto. Isso é chamado de apelidamento em outras linguagens. Geralmente, esta característica não é muito apreciada, e pode ser ignorada com segurança ao lidar com tipos imutáveis (números, strings, tuplas). Entretanto, apelidamento pode ter um efeito surpreendente na semântica do código Python envolvendo objetos mutáveis como listas, dicionários e a maioria dos outros tipos. Isso pode ser usado em benefício do programa, porque os apelidos funcionam de certa forma como ponteiros. Por exemplo, passar um objeto como argumento é barato, pois só um ponteiro é passado na implementação; e se uma função modifica um objeto passado como argumento, o invocador verá a mudança — isso elimina a necessidade de ter dois mecanismos de passagem de parâmetros como em Pascal.

## 2. Escopos e espaços de nomes (NameSpaces) do Python

Antes de introduzir classes, é preciso falar das regras de escopo em Python. Definições de classe fazem alguns truques com espaços de nomes. Portanto, primeiro é preciso entender claramente como escopos e espaços de nomes funcionam, para entender o que está acontecendo. Esse conhecimento é muito útil para qualquer programador Python avançado.

Vamos começar com algumas definições.

Um espaço de nomes é um mapeamento que associa nomes a objetos. Atualmente, são implementados como dicionários em Python, mas isso não é perceptível (a não ser pelo desempenho), e pode mudar no futuro.<br>
Exemplos de espaços de nomes são :
- o conjunto de nomes pré-definidos (funções como [abs()](https://docs.python.org/pt-br/3/library/functions.html#abs) e as exceções pré-definidas);
- nomes globais em um módulo;
- e nomes locais na invocação de uma função.

De certa forma, os atributos de um objeto também formam um espaço de nomes. O mais importante é saber que não existe nenhuma relação entre nomes em espaços de nomes distintos. Por exemplo, dois módulos podem definir uma função de nome maximize sem confusão — usuários dos módulos devem prefixar a função com o nome do módulo, para evitar colisão.

A propósito, utilizo a palavra `atributo` para qualquer nome depois de um ponto. Na expressão `z.real`, por exemplo, `real` é um atributo do objeto `z`. Estritamente falando, referências para nomes em módulos são atributos: na expressão `modname.funcname`, `modname` é um objeto módulo e `funcname` é um de seus atributos. Neste caso, existe um mapeamento direto entre os atributos de um módulo e os nomes globais definidos no módulo: eles compartilham o mesmo espaço de nomes!

Atributos podem ser somente *leitura* ou para *leitura* e *escrita*. No segundo caso, é possível atribuir um novo valor ao atributo. Atributos de módulos são passíveis de atribuição: você pode escrever `modname.the_answer = 42`. Atributos que aceitam escrita também podem ser apagados através da instrução [del](https://docs.python.org/pt-br/3/reference/simple_stmts.html#del). Por exemplo, `del modname.the_answer` removerá o atributo `the_answer` do objeto referenciado por `modname`.

Espaços de nomes são criados em momentos diferentes e possuem diferentes ciclos de vida. O espaço de nomes que contém os nomes **built-in** é criado quando o interpretador inicializa e nunca é removido. O espaço de nomes global de um módulo é criado quando a definição do módulo é lida, e normalmente duram até a finalização do interpretador. Os comandos executados pela invocação do interpretador, pela leitura de um script como programa principal, ou interativamente, são parte do módulo chamado [__main__](https://docs.python.org/pt-br/3/library/__main__.html#module-__main__), e portanto possuem seu próprio espaço de nomes. (Os nomes **built-in** possuem seu próprio espaço de nomes no módulo chamado [builtins](https://docs.python.org/pt-br/3/library/builtins.html#module-builtins)).

O espaço de nomes local de uma função é criado quando a função é invocada, e apagado quando a função retorna ou levanta uma exceção que não é tratada na própria função. (Na verdade, uma forma melhor de descrever o que realmente acontece é que o espaço de nomes local é “esquecido” quando a função termina.) Naturalmente, cada invocação recursiva de uma função tem seu próprio espaço de nomes.

Um escopo é uma região textual de um programa Python onde um espaço de nomes é diretamente acessível. Aqui, *"diretamente acessível"* significa que uma referência sem um prefixo qualificador permite o acesso ao nome.

Ainda que escopos sejam determinados estaticamente, eles são usados dinamicamente. A qualquer momento durante a execução, existem 3 ou 4 escopos aninhados cujos espaços de nomes são diretamente acessíveis:
- o escopo mais interno, que é acessado primeiro, contem os nomes locais;
- os escopos das funções que envolvem a função atual, que são acessados a partir do escopo mais próximo, contêm nomes não-locais, mas também não-globais;
- o penúltimo escopo contém os nomes globais do módulo atual;
- e o escopo mais externo (acessado por último) contém os nomes das funções embutidas e demais objetos pré-definidos do interpretador;

Se um nome é declarado no escopo global, então todas as referências e atribuições de valores vão diretamente para o penúltimo escopo, que contém os nomes globais do módulo. Para alterar variáveis declaradas fora do escopo mais interno, a instrução [nonlocal](https://docs.python.org/pt-br/3/reference/simple_stmts.html#nonlocal) pode ser usada; caso contrário, todas essas variáveis serão apenas para leitura (a tentativa de atribuir valores a essas variáveis simplesmente criará uma nova variável local, no escopo interno, não alterando nada na variável de nome idêntico fora dele).

Normalmente, o escopo local referencia os nomes locais da função corrente no texto do programa. Fora de funções, o escopo local referencia os nomes do escopo global: espaço de nomes do módulo. Definições de classes adicionam um outro espaço de nomes ao escopo local.

É importante perceber que escopos são determinados estaticamente, pelo texto do código-fonte: o escopo global de uma função definida em um módulo é o espaço de nomes deste módulo, sem importar de onde ou por qual apelido a função é invocada. Por outro lado, a busca de nomes é dinâmica, ocorrendo durante a execução. Porém, a evolução da linguagem está caminhando para uma resolução de nomes estática, em “tempo de compilação”, portanto não conte com a resolução dinâmica de nomes! (De fato, variáveis locais já são resolvidas estaticamente).

Uma peculiaridade especial do Python é que – se nenhuma instrução [global](https://docs.python.org/pt-br/3/reference/simple_stmts.html#global) ou [nonlocal](https://docs.python.org/pt-br/3/reference/simple_stmts.html#nonlocal) estiver em vigor – as atribuições de nomes sempre entram no escopo mais interno. As atribuições não copiam dados — elas apenas vinculam nomes aos objetos. O mesmo vale para exclusões: a instrução `del x` remove a ligação de `x` do espaço de nomes referenciado pelo escopo local. De fato, todas as operações que introduzem novos nomes usam o escopo local: em particular, instruções [import](https://docs.python.org/pt-br/3/reference/simple_stmts.html#import) e definições de funções ligam o módulo ou o nome da função no escopo local.

A instrução [global](https://docs.python.org/pt-br/3/reference/simple_stmts.html#global) pode ser usada para indicar que certas variáveis residem no escopo global ao invés do local; a instrução [nonlocal](https://docs.python.org/pt-br/3/reference/simple_stmts.html#nonlocal) indica que variáveis particulares estão em um espoco mais interno e devem ser recuperadas lá.

### 2.1. Exemplo de escopos e espaço de nomes

Este é um exemplo que demonstra como se referir aos diferentes escopos e aos espaços de nomes, e como [global](https://docs.python.org/pt-br/3/reference/simple_stmts.html#global) e [nonlocal](https://docs.python.org/pt-br/3/reference/simple_stmts.html#nonlocal) pode afetar ligação entre as variáveis.

In [ ]:
def teste_escopo():
    def cria_local():
        spam = "local spam"

    def cria_nonlocal():
        nonlocal spam
        spam = "nonlocal spam"

    def cria_global():
        global spam
        spam = "global spam"

    spam = "teste spam"
    cria_local()
    print("Depois da atribuição local :", spam)
    cria_nonlocal()
    print("Depois da atribuição nonlocal :", spam)
    cria_global()
    print("Depois da atribuição global :", spam)

teste_escopo()
print("No escopo global :", spam)

A saída do código de exemplo é :

`Depois da atribuição local : teste spam`<br>
`Depois da atribuição nonlocal : nonlocal spam`<br>
`Depois da atribuição global : nonlocal spam`<br>
`No escopo global : global spam`

Observe como uma atribuição *local* (que é o padrão) não altera o vínculo de teste_escopo a spam. A instrução [nonlocal](https://docs.python.org/pt-br/3/reference/simple_stmts.html#nonlocal) mudou o vínculo de escopo_teste de spam e a atribuição [global](https://docs.python.org/pt-br/3/reference/simple_stmts.html#global) alterou a ligação para o nível do módulo.

Você também pode ver que não havia nenhuma ligação anterior para spam antes da atribuição [global](https://docs.python.org/pt-br/3/reference/simple_stmts.html#global).

## 3. Uma primeira olhada nas classes

Classes introduzem novidades sintáticas, três novos tipos de objetos, e também alguma semântica nova.

### 3.1. Sintaxe da definição de classe

A forma mais simples de definir uma classe é :

In [ ]:
class NomeClasse:
    ...

Definições de classe, assim como definições de função (instruções [def](https://docs.python.org/pt-br/3/reference/compound_stmts.html#def)), precisam ser executadas antes que tenham qualquer efeito. (Você pode colocar uma definição de classe dentro do teste condicional de um [if](https://docs.python.org/pt-br/3/reference/compound_stmts.html#if) ou dentro de uma função).

Na prática, as instruções dentro da definição de classe geralmente serão definições de funções, mas outras instruções são permitidas, e às vezes são bem úteis — voltaremos a este tema depois. Definições de funções dentro da classe normalmente têm um forma peculiar de lista de argumentos, determinada pela convenção de chamada a métodos — isso também será explicado mais tarde.

Quando se inicia a definição de classe, um novo espaço de nomes (namespaces) é criado, e usado como escopo local — assim, todas atribuições a variáveis locais ocorrem nesse espaço de nomes. Em particular, funções definidas aqui são vinculadas a nomes nesse escopo.

Quando uma definição de classe é finalizada normalmente (até o fim), um objeto classe é criado. Este objeto encapsula o conteúdo do espaço de nomes criado pela definição da classe. O escopo local que estava vigente antes da definição da classe é reativado, e o objeto classe é vinculado ao identificador da classe nesse escopo (`NomeClasse` no exemplo).

### 3.2. Objetos da Classe

Objetos classe suportam dois tipos de operações: `referências a atributos` e `instanciação`.

`Referências a atributos` de classe utilizam a sintaxe padrão utilizada para quaisquer referências a atributos em Python: `obj.nome`. Nomes de atributos válidos são todos os nomes presentes dentro do espaço de nomes da classe, quando o objeto classe foi criado. Portanto, se a definição de classe tem esta forma :

In [ ]:
class MinhaClasse:
    """Um simples exemplo de classe"""
    n = 12345

    def funcao(self):
        return 'Olá, Mundo!'

então, `MinhaClasse.n` e `MinhaClasse.funcao` são referências a atributo válidas, retornando, respectivamente, um inteiro e um objeto função. Atributos de classe podem receber valores, pode-se modificar o valor de `MinhaClasse.n` num atribuição. `__doc__` também é um atributo válido da classe, retornando a documentação associada : "Um simples exemplo de classe".

Para `instanciar` uma classe, usa-se a mesma sintaxe de invocar uma função. Apenas finja que o objeto classe do exemplo é uma função sem parâmetros, que devolve uma nova instância da classe. Por exemplo (assumindo a classe acima) :

In [ ]:
x = MinhaClasse()

cria uma nova `instância` da classe e atribui o objeto resultante à variável local `x`.

A operação de instanciação (“invocar” um objeto classe) cria um objeto vazio. Muitas classes preferem criar novos objetos com um estado inicial predeterminado. Para tanto, a classe pode definir um método especial chamado `__init__()`, assim :

In [ ]:
def __init__(self):
    self.dados = []

quando uma classe define um método `__init__()`, o processo de instanciação automaticamente invoca `__init__()` sobre a instância recém criada. Em nosso exemplo, uma nova instância já inicializada pode ser obtida desta maneira :

In [ ]:
x = MinhaClasse()

Naturalmente, o método `__init__()` pode ter parâmetros para maior flexibilidade. Neste caso, os argumentos fornecidos na invocação da classe serão passados para o método `__init__()`. Por exemplo :

In [ ]:
class Complexo:
    def __init__(self, parte_real, parte_imaginaria):
        self.real = parte_real
        self.imag = parte_imaginaria

num_comp = Complexo(3.0, -4.5)
print(num_comp.real, num_comp.imag)

### 3.3. Objetos Instância

Agora o que podemos fazer com objetos de instância? As únicas operações compreendidas por objetos de instância são os atributos de referência. Existem duas maneiras válidas para nomear atributos: atributos de dados e métodos.

Atributos de dados correspondem a "variáveis de instância" em Smalltalk, e a "membros de dados" em C++. Atributos de dados não precisam ser declarados. Assim como variáveis locais, eles passam a existir na primeira vez em que é feita uma atribuição. Por exemplo, se `n` é uma instância da `MinhaClasse` criada acima, o próximo trecho de código irá exibir o valor `16`, sem deixar nenhum rastro :

In [ ]:
class MinhaClasse:
    ...

n = MinhaClasse()
n.contador = 1
while n.contador < 10:
    n.contador = n.contador * 2
print(n.contador)
del n.contador

O outro tipo de referências a atributos de instância é o "método". Um método é uma função que "pertence" a um objeto instância. (Em Python, o termo método não é aplicado exclusivamente a instâncias de classes definidas pelo usuário: outros tipos de objetos também podem ter métodos. Por exemplo, listas possuem os métodos append, insert, remove, sort, entre outros. Porém, na discussão a seguir, usaremos o termo método apenas para se referir a métodos de classes definidas pelo usuário. Seremos explícitos ao falar de outros métodos).

Nomes de métodos válidos de uma instância dependem de sua classe. Por definição, cada atributo de uma classe que é uma função corresponde a um método das instâncias. Em nosso exemplo, `x.funcao` é uma referência de método válida já que `MinhaClasse.funcao` é uma função, enquanto `x.n` não é, já que `MinhaClasse.n` não é uma função. Entretanto, `x.funcao` não é o mesmo que `MinhaClasse.funcao`. A referência `x.funcao` acessa um objeto método e a `MinhaClasse.funcao` acessa um objeto função.

### 3.4. Objetos Método

Normalmente, um método é chamado imediatamente após ser referenciado.

In [ ]:
class MinhaClasse:
    """Um simples exemplo de classe"""
    n = 12345

    def funcao(self):
        return 'Olá, Mundo!'

x.funcao()

No exemplo `MinhaClasse`, o resultado da expressão acima será a string 'Olá, Mundo!'. No entanto, não é obrigatório invocar o método imediatamente: como `x.funcao` é também um objeto ele pode ser atribuído a uma variável e invocado depois. O exemplo :

In [ ]:
class MinhaClasse:
    """Um simples exemplo de classe"""
    n = 12345

    def funcao(self):
        return 'Olá, Mundo!'

x_funcao = x.funcao
while True:
    print(x_funcao())

exibirá o texto "Olá, Mundo!" até o mundo acabar.

O que ocorre precisamente quando um método é invocado? Você deve ter notado que `x.funcao()` foi chamado sem nenhum argumento, porém a definição da função `funcao()` especificava um argumento. O que aconteceu com esse argumento? Certamente Python levanta uma exceção quando uma função que declara um argumento é invocada sem nenhum argumento — mesmo que o argumento não seja usado no corpo da função…

Na verdade, pode-se supor a resposta: a particularidade sobre os métodos é que o objeto da instância é passado como o primeiro argumento da função. Em nosso exemplo, a chamada `x.funcao()` é exatamente equivalente a `MinhaClasse.funcao(x)`. Em geral, chamar um método com uma lista de *n argumentos* é equivalente a chamar a função correspondente com uma lista de argumentos que é criada inserindo o objeto de instância do método antes do primeiro argumento.

Se você ainda não entende como os métodos funcionam, dê uma olhada na implementação para esclarecer as coisas. Quando um atributo de uma instância, não relacionado a dados, é referenciado, a classe da instância é pesquisada. Se o nome é um atributo de classe válido, e é o nome de uma função, um método é criado, empacotando a instância e a função, que estão juntos num objeto abstrato: este é o método. Quando o método é invocado com uma lista de argumentos, uma nova lista de argumentos é criada inserindo a instância na posição 0 da lista. Finalmente, o objeto função — empacotado dentro do objeto método — é invocado com a nova lista de argumentos.

### 3.5. Variáveis de Classe e Instância

De forma geral, variáveis de instância são variáveis que indicam dados que são únicos a cada instância individual, e variáveis de classe são variáveis de atributos e de métodos que são comuns a todas as instâncias de uma classe.

In [ ]:
class Cao:
    tipo = 'canino'     # variável de classe compartilhado por todas as instâncias
    def __init__(self, nome):
        self.nome = nome     # variável de instância uníco para cada instância

cao_fido = Cao('Fido')
cao_rex = Cao('Rex')
print(cao_fido.tipo) # compartilhado por todos os cães
print(cao_rex.tipo)  # compartilhado por todos os cães
print(cao_fido.nome) # único para cao_fido
print(cao_rex.nome)  # único para cao_rex

Como vimos em "Uma palavra sobre nomes e objetos", dados compartilhados podem causar efeitos inesperados quando envolvem objetos ([mutáveis](https://docs.python.org/pt-br/3/glossary.html#term-mutable)), como listas ou dicionários. Por exemplo, a lista **truques** do código abaixo não deve ser usada como variável de classe, pois assim seria compartilhada por todas as instâncias de Cao :

In [ ]:
class Cao:
    truques = []     # uso incorreto de uma variável de classe
    def __init__(self, nome):
        self.nome = nome   
    def novo_truque(self, truque):
        self.truques.append(truque)

cao_fido = Cao('Fido')
cao_rex = Cao('Rex')
cao_fido.novo_truque('rolar')
cao_rex.novo_truque('sentar')
print(cao_fido.truques) # inesperadamente compartilhado com todos os cães

Em vez disso, o modelo correto da classe deve usar uma variável de instância :

In [ ]:
class Cao:
    def __init__(self, nome):
        self.nome = nome   
        self.truques = []     # cria uma nova lista para cada cão
    def novo_truque(self, truque):
        self.truques.append(truque)

cao_fido = Cao('Fido')
cao_rex = Cao('Rex')
cao_fido.novo_truque('rolar')
cao_rex.novo_truque('sentar')
print(cao_fido.truques)
print(cao_rex.truques)

## 4. Observações Aleatórias

Se um mesmo nome de atributo ocorre tanto na instância quanto na classe, a busca pelo atributo prioriza a instância :

In [ ]:
class Armazem:
    proposito = 'armazenar'
    regiao = 'sul'

a1 = Armazem()
print(a1.proposito, a1.regiao)

a2 = Armazem()
a2.regiao = 'sudeste'
print(a2.proposito, a2.regiao)

Atributos de dados podem ser referenciados por métodos da própria instância, bem como por qualquer outro usuário do objeto (também chamados “clientes” do objeto). Em outras palavras, classes não servem para implementar tipos puramente abstratos de dados. De fato, nada em Python torna possível assegurar o encapsulamento de dados — tudo é baseado em convenção. (Por outro lado, a implementação de Python, escrita em C, pode esconder completamente detalhes de um objeto e controlar o acesso ao objeto, se necessário; isto pode ser utilizado por extensões de Python escritas em C).

Clientes devem utilizar atributos de dados com cuidado, pois podem bagunçar invariantes assumidas pelos métodos ao esbarrar em seus atributos de dados. Note que clientes podem adicionar atributos de dados a suas próprias instâncias, sem afetar a validade dos métodos, desde que seja evitado o conflito de nomes. Novamente, uma convenção de nomenclatura poupa muita dor de cabeça.

Não existe atalho para referenciar atributos de dados (ou outros métodos!) de dentro de um método. Isso aumenta a legibilidade dos métodos: não há como confundir variáveis locais com variáveis da instância quando lemos rapidamente um método.

Frequentemente, o primeiro argumento de um método é chamado `self`. Isso não passa de uma convenção: o identificador `self` não é uma palavra reservada nem possui qualquer significado especial em Python. Mas note que, ao seguir essa convenção, seu código se torna legível por uma grande comunidade de desenvolvedores Python e é possível que alguma IDE dependa dessa convenção para analisar seu código.

Qualquer objeto função que é atributo de uma classe, define um método para as instâncias dessa classe. Não é necessário que a definição da função esteja textualmente embutida na definição da classe. Atribuir um objeto função a uma variável local da classe é válido. Por exemplo :

In [ ]:
def f1(self, x, y):
    return min(x, x+y)

class C:
    f = f1

    def g(self):
        return 'Olá, mundo!'
    
    h = g

Agora `f`, `g` e `h` são todos atributos da classe C que referenciam funções, e consequentemente são todos métodos de instâncias da classe C, onde `h` é exatamente equivalente a `g`. No entanto, essa prática serve apenas para confundir o leitor do programa.

Métodos podem invocar outros métodos usando atributos de método do argumento `self`.

In [ ]:
class Sacola:
    def __init__(self):
        self.dados = []
    
    def adiciona(self, x):
        self.dados.append(x)
    
    def adiciona_duas_vezes(self, x):
        self.adiciona(x)
        self.adiciona(x)

Métodos podem referenciar nomes globais da mesma forma que funções comuns. O escopo global associado a um método é o módulo contendo sua definição na classe (a classe propriamente dita nunca é usada como escopo global!). Ainda que seja raro justificar o uso de dados globais em um método, há diversos usos legítimos do escopo global. Por exemplo, funções e módulos importados no escopo global podem ser usados por métodos, bem como as funções e classes definidas no próprio escopo global. Provavelmente, a classe contendo o método em questão também foi definida neste escopo global. Na próxima seção veremos razões pelas quais um método pode querer referenciar sua própria classe.

Cada valor é um objeto e, portanto, tem uma *classe* (também chamada de *tipo*). Ela é armazenada como `object.__class__`.

## 5. Atributos

Em Python, os atributos de uma classe podem ser públicos ou privados.

Atributos públicos são aqueles que não possuem nenhum prefixo ou sufixo especial em seus nomes. Isso significa que eles podem ser acessados e modificados por qualquer objeto da classe, bem como por qualquer objeto que herde da classe.

Já os atributos privados são aqueles que possuem um prefixo `__` (duas underlines) no início do seu nome. Esses atributos só podem ser acessados e modificados diretamente dentro da classe onde eles foram declarados. Acessá-los de fora da classe é considerado uma violação de encapsulamento e pode causar problemas no funcionamento do código.

É importante notar que, embora esses atributos possam ser acessados e modificados diretamente dentro da classe, é geralmente recomendado utilizar métodos getters e setters para fazer isso de forma controlada e segura.

### 5.1. Atributos Públicos

Em Python, os atributos de uma classe são considerados públicos quando não possuem nenhum prefixo ou sufixo especial em seus nomes. Isso significa que eles podem ser acessados e modificados diretamente por qualquer objeto da classe, bem como por qualquer objeto que herde da classe.

#### 5.1.1. Básico

Considere a seguinte classe `Personagem` :

In [ ]:
class Personagem:
    def __init__(self, nome, classe, nivel, vida, mana):
        self.nome = nome
        self.classe = classe
        self.nivel = nivel
        self.vida = vida
        self.mana = mana


Nesta classe, os atributos `nome`, `classe`, `nivel`, `vida` e `mana` são públicos, pois não possuem nenhum prefixo ou sufixo especial em seus nomes. Isso significa que podemos acessá-los diretamente, como no exemplo abaixo :

In [ ]:
guerreiro = Personagem("Conan", "Guerreiro", 10, 100, 0)
print(guerreiro.nome) # Imprime "Conan"
print(guerreiro.classe) # Imprime "Guerreiro"
print(guerreiro.nivel) # Imprime 10

Além disso, também podemos modificar esses atributos diretamente, como no exemplo abaixo :

In [ ]:
guerreiro.vida = 80
print(guerreiro.vida) # Imprime 80

Exemplo 1 : criando uma classe `Monstro` com atributos públicos :

In [ ]:
class Monstro:
    def __init__(self, nome, tipo, vida, dano):
        self.nome = nome
        self.tipo = tipo
        self.vida = vida
        self.dano = dano

goblin = Monstro("Goblin", "humanoide", 20, 5)
print(goblin.nome) # Imprime "Goblin"
print(goblin.tipo) # Imprime "humanoide"

Nesta classe, os atributos `nome`, `tipo`, `vida` e `dano` são públicos, o que significa que podemos acessá-los diretamente.

Exemplo 2 : criando uma classe `Item` com atributos públicos :

In [ ]:
class Item:
    def __init__(self, nome, tipo, preco):
        self.nome = nome
        self.tipo = tipo
        self.preco = preco

espada = Item("Espada Longa", "arma", 100)
print(espada.nome) # Imprime "Espada Longa"
print(espada.tipo) # Imprime "arma"

Nesta classe, os atributos `nome`, `tipo` e `preco` são públicos, o que significa que podemos acessá-los diretamente.

#### 5.1.2. Atributos Públicos com Métodos

Aqui estão alguns exemplos adicionais de como podemos utilizar atributos públicos em classes com métodos :

Exemplo 1 : criando uma classe `Personagem` com atributos públicos e métodos `atacar` e `curar` :

In [ ]:
class Personagem:
    def __init__(self, nome, classe, nivel, vida, mana):
        self.nome = nome
        self.classe = classe
        self.nivel = nivel
        self.vida = vida
        self.mana = mana

    def atacar(self, alvo):
        alvo.vida -= self.dano

    def curar(self):
        self.vida += 10
        self.mana -= 5

Nesta classe, os atributos `nome`, `classe`, `nivel`, `vida` e `mana` são públicos, o que significa que podemos acessá-los diretamente, como abaixo :

In [ ]:
guerreiro = Personagem("Conan", "Guerreiro", 10, 100, 0)
print(guerreiro.nome) # Imprime "Conan"
guerreiro.atacar(goblin) # Diminui vida do goblin
guerreiro.curar() # Aumenta vida do guerreiro

Exemplo 2: criando uma classe `Item` com atributos públicos e método `usar` :

In [ ]:
class Item:
    def __init__(self, nome, tipo, preco):
        self.nome = nome
        self.tipo = tipo
        self.preco = preco

    def usar(self, personagem):
        if self.tipo == "arma":
            personagem.dano += 10
        elif self.tipo == "poção":
            personagem.vida += 20

espada = Item("Espada Longa", "arma", 100)
print(espada.nome) # Imprime "Espada Longa"
espada.usar(guerreiro) # Aumenta dano do guerreiro

Nesta classe, os atributos `nome`, `tipo` e `preco` são públicos, o que significa que podemos acessá-los diretamente.

### 5.1. Atributos Privados

Em Python, os atributos de uma classe podem ser declarados como privados adicionando um prefixo `__` (duas underlines) no início do seu nome. Esses atributos só podem ser acessados e modificados diretamente dentro da classe onde eles foram declarados. Acessá-los de fora da classe é considerado uma violação de encapsulamento e pode causar problemas no funcionamento do código.

Isso pode ser útil para garantir que certos valores não possam ser modificados diretamente, mas apenas através de métodos específicos.

Por exemplo, considere a seguinte classe `Personagem` :

In [ ]:
class Personagem:
    def __init__(self, nome, classe, nivel, vida, mana):
        self.__nome = nome
        self.__classe = classe
        self.__nivel = nivel
        self.__vida = vida
        self.__mana = mana

Nesta classe, os atributos `__nome`, `__classe`, `__nivel`, `__vida` e `__mana` são privados, pois possuem o prefixo "" em seus nomes. Isso significa que não podemos acessá-los diretamente, como no exemplo abaixo :

In [ ]:
guerreiro = Personagem("Conan", "Guerreiro", 10, 100, 0)
print(guerreiro.__nome) # resultará em um erro, pois não é possível acessar diretamente

Para acessar esses atributos, é necessário criar métodos `getters` e `setters` específicos para a classe, como no exemplo abaixo :

In [ ]:
class Personagem:
    def __init__(self, nome, classe, nivel, vida, mana):
        self.__nome = nome
        self.__classe = classe
        self.__nivel = nivel
        self.__vida = vida
        self.__mana = mana

    def get_nome(self):
        return self.__nome

    def set_nome(self, novo_nome):
        self.__nome = novo_nome

Dessa forma, é possível acessar e modificar os atributos privados através dos métodos específicos :

In [ ]:
guerreiro = Personagem("Conan", "Guerreiro", 10, 100, 0)
print(guerreiro.get_nome()) # Imprime "Conan"
guerreiro.set_nome("Arnold")
print(guerreiro.get_nome()) # Imprime "Arnold"

Exemplo 1 : criando uma classe `Monstro` com atributos privados e métodos `get_vida` e `set_vida` :

In [ ]:
class Monstro:
    def __init__(self, nome, tipo, vida, dano):
        self.__nome = nome
        self.__tipo = tipo
        self.__vida = vida
        self.__dano = dano

    def get_vida(self):
        return self.__vida

    def set_vida(self, nova_vida):
        self.__vida = nova_vida

goblin = Monstro("Goblin", "humanoide", 20, 5)
print(goblin.get_vida()) # Imprime 20
goblin.set_vida(15)
print(goblin.get_vida()) # Imprime 15


Nesta classe, os atributos `__nome`, `__tipo`, `__vida` e `__dano` são privados, o que significa que não podemos acessá-los diretamente. O atributo `__vida` só pode ser acessado apenas através dos métodos `get_vida` e `set_vida`.

Exemplo 2 : criando uma classe `Item` com atributos privados e método `get_preco` :

In [ ]:
class Item:
    def __init__(self, nome, tipo, preco):
        self.nome = nome
        self.tipo = tipo
        self.__preco = preco

    def get_preco(self):
        return self.__preco

espada = Item("Espada Longa", "arma", 100)
print(espada.get_preco()) # Imprime 100

Nesta classe, o atributo `__preco` é privado, o que significa que não podemos acessá-lo diretamente, mas apenas através do método `get_preco`.

## 6. Herança

Obviamente, uma característica da linguagem não seria digna do nome "classe" se não suportasse herança. A sintaxe para uma classe derivada é assim :

In [ ]:
class NomeClasseDerivada(NomeClasseBase):
    ...

O identificador `NomeClasseBase` deve estar definido no escopo que contém a definição da classe derivada. No lugar do nome da classe base, também são aceitas outras expressões. Isso é muito útil, por exemplo, quando a classe base é definida em outro módulo :

In [ ]:
class NomeClasseDerivada(nome_modulo.NomeClasseBase):
    ...

A execução de uma definição de classe derivada procede da mesma forma que a de uma classe base. Quando o objeto classe é construído, a classe base é lembrada. Isso é utilizado para resolver referências a atributos. Se um atributo requisitado não for encontrado na classe, ele é procurado na classe base. Essa regra é aplicada recursivamente se a classe base por sua vez for derivada de outra.

Não há nada de especial sobre instanciação de classes derivadas: `NomeClasseDerivada()` cria uma nova instância da classe. Referências a métodos são resolvidas da seguinte forma: o atributo correspondente é procurado através da cadeia de classes base, e referências a métodos são válidas se essa procura produzir um objeto função.

Classes derivadas podem sobrescrever métodos das suas classes base. Uma vez que métodos não possuem privilégios especiais quando invocam outros métodos no mesmo objeto, um método na classe base que invoca um outro método da mesma classe base pode, efetivamente, acabar invocando um método sobreposto por uma classe derivada. (Para programadores C++ isso significa que todos os métodos em Python são realmente **virtuais**).

Um método sobrescrito em uma classe derivada, de fato, pode querer estender, em vez de simplesmente substituir, o método da classe base, de mesmo nome. Existe uma maneira simples de chamar diretamente o método da classe base: apenas chame `NomeClasseBase.nome_metodo(self, argumentos)`. Isso é geralmente útil para os clientes também. (Note que isto só funciona se a classe base estiver acessível como `NomeClasseBase` no escopo global).

Python tem duas funções **built-in** que trabalham com herança :
- use `isinstance()` para verificar o tipo de uma instância : `isinstance(obj, int)` será **True** somente se `obj.__class__` é a classe [int](https://docs.python.org/pt-br/3/library/functions.html#int) ou alguma classe derivada de [int](https://docs.python.org/pt-br/3/library/functions.html#int).
- use `issubclass()` para verificar herança entre classes : `issubclass(bool, int)` é **True** porque [bool](https://docs.python.org/pt-br/3/library/functions.html#bool) é uma subclasse de [int](https://docs.python.org/pt-br/3/library/functions.html#int). Porém, `issubclass(float, int)` é False porque [float](https://docs.python.org/pt-br/3/library/functions.html#float) não é uma subclasse de [int](https://docs.python.org/pt-br/3/library/functions.html#int).

### 5.1. Herança Múltipla

Python também suporta uma forma de herança múltipla. Uma definição de classe com várias classes bases tem esta forma :

In [ ]:
class NomeClasseDerivada(Base1, Base2, Base3):
    ...

Para a maioria dos casos mais simples, pense na pesquisa de atributos herdados de uma classe pai como o primeiro nível de profundidade, da esquerda para a direita, não pesquisando duas vezes na mesma classe em que há uma sobreposição na hierarquia. Assim, se um atributo não é encontrado em `NomeClasseDerivada`, é procurado em `Base1`, depois, recursivamente, nas classes base de `Base1`, e se não for encontrado lá, é pesquisado em `Base2` e assim por diante.

De fato, é um pouco mais complexo que isso; a ordem de resolução de métodos muda dinamicamente para suportar chamadas cooperativas para [super()](https://docs.python.org/pt-br/3/library/functions.html#super). Essa abordagem é conhecida em outras linguagens de herança múltipla como chamar-o-próximo-método, e é mais poderosa que a chamada à função **super**, encontrada em linguagens de herança única.

A ordenação dinâmica é necessária porque todos os casos de herança múltipla exibem um ou mais relacionamentos de diamante (em que pelo menos uma das classes pai pode ser acessada por meio de vários caminhos da classe mais inferior). Por exemplo, todas as classes herdam de [object](https://docs.python.org/pt-br/3/library/functions.html#object), portanto, qualquer caso de herança múltipla fornece mais de um caminho para alcançar [object](https://docs.python.org/pt-br/3/library/functions.html#object). Para evitar que as classes base sejam acessadas mais de uma vez, o algoritmo dinâmico lineariza a ordem de pesquisa, de forma a preservar a ordenação da esquerda para a direita, especificada em cada classe, que chama cada pai apenas uma vez, e que é monotônica (significando que uma classe pode ser subclassificada sem afetar a ordem de precedência de seus pais). Juntas, essas propriedades tornam possível projetar classes confiáveis e extensíveis com herança múltipla. Para mais detalhes, veja [aqui](https://www.python.org/download/releases/2.3/mro/).

## 7. Métodos Estáticos

O `@staticmethod` das classes são métodos que não precisam de uma instância da classe para serem chamados. Eles são definidos usando a decorator `@staticmethod` antes da declaração do método. Eles podem acessar apenas variáveis e métodos estáticos da classe.

Aqui está um exemplo de como usar `@staticmethod` :

In [ ]:
class MinhaClasse:
    var_estatica = "variavel estatica"

    @staticmethod
    def metodo_estatico():
        print(MinhaClasse.var_estatica)

MinhaClasse.metodo_estatico()  # imprime "variavel estatica"

Neste exemplo, o método `metodo_estatico` é definido como estático usando o decorador `@staticmethod`. Ele acessa a variável de classe `var_estatica` e imprime seu valor. Ele pode ser chamado diretamente a partir da classe, sem a necessidade de criar uma instância da classe.

É importante notar que, diferente de métodos normais, os métodos estáticos não têm acesso às variáveis de instância e métodos da classe. Eles só podem acessar variáveis e métodos estáticos.

In [ ]:
class MinhaClasse:
    var_estatica = "variavel estatica"
    def __init__(self):
        self.var_instancia = "variavel de instancia"

    @staticmethod
    def metodo_estatico():
        print(MinhaClasse.var_estatica)
        print(self.var_instancia)  # gera erro, pois self não é definido


objeto = MinhaClasse()
objeto.metodo_estatico()  

O exemplo acima gera um erro, pois o método estático não tem acesso à variáveis de instância.

`@staticmethod` é um decorator em Python que é usado para definir um método estático dentro de uma classe. Um método estático é um método que não precisa de uma instância de uma classe para ser chamado. Em vez disso, eles são chamados diretamente a partir da classe.

Os métodos estáticos são úteis quando você precisa de uma função que não precisa acessar as variáveis de instância ou os métodos da classe. Eles só têm acesso às variáveis e métodos estáticos da classe.

Aqui está um exemplo de como usar `@staticmethod` :

In [ ]:
class MinhaClasse:
    var_estatica = "variavel estatica"

    @staticmethod
    def metodo_estatico():
        print(MinhaClasse.var_estatica)

MinhaClasse.metodo_estatico()  # imprime "variavel estatica"

Neste exemplo, o método `metodo_estatico` é definido como estático usando o decorador `@staticmethod`. Ele acessa a variável de classe `var_estatica` e imprime seu valor. Ele pode ser chamado diretamente a partir da classe, sem a necessidade de criar uma instância da classe.

Além disso, os métodos estáticos são geralmente usados para funções relacionadas à classe, como cálculos, conversões de unidades, etc., que não precisam de acesso aos dados de instância.

É importante notar que, ao contrário dos métodos normais, os métodos estáticos não têm acesso às variáveis de instância e métodos da classe. Eles só podem acessar variáveis e métodos estáticos.

Ao chamar um método estático, não é preciso criar uma instância da classe, o que pode ser útil para economizar recursos. Além disso, os métodos estáticos podem ser chamados diretamente a partir da classe, sem a necessidade de criar uma instância.

Aqui temos um exemplo de como usar `@staticmethod` para criar uma calculadora simples :

In [ ]:
class Calculadora:
    @staticmethod
    def soma(x, y):
        return x + y

    @staticmethod
    def subtracao(x, y):
        return x - y

    @staticmethod
    def multiplicacao(x, y):
        return x * y

    @staticmethod
    def divisao(x, y):
        if y == 0:
            raise ValueError("Não é possível dividir por zero")
        return x / y

# exemplos de uso
print(Calculadora.soma(10, 5))  # imprime 15
print(Calculadora.subtracao(10, 5))  # imprime 5
print(Calculadora.multiplicacao(10, 5))  # imprime 50
print(Calculadora.divisao(10, 5))  # imprime 2.0

Neste exemplo, a classe `Calculadora` define quatro métodos estáticos: `soma`, `subtracao`, `multiplicacao` e `divisao`. Cada um deles é definido como estático usando o decorador `@staticmethod`. Esses métodos podem ser chamados diretamente a partir da classe `Calculadora`, sem a necessidade de criar uma instância dela.

Os exemplos mostram o uso desses métodos, passando dois argumentos para cada um deles, e imprimindo o resultado.

É importante observar que, nesse exemplo, é tratado o caso de divisão por zero, que é uma operação inválida e gera erro, por isso é lançado um ValueError com uma mensagem de erro.

A seguir, temos mais exemplos de `@staticmethods` :

In [ ]:
class MinhaClasse:
    x = [1, 2, 3]

    @staticmethod
    def metodo_estatico(a, b):
        return a + b


# Utilizando o método estático
print(MinhaClasse.metodo_estatico(1, 2)) # 3

# Criando uma instância da classe
obj = MinhaClasse()

# Utilizando o método estático através da instância
print(obj.metodo_estatico(1, 2)) # 3

No exemplo acima, o método `metodo_estatico` é um método estático que soma dois números e retorna o resultado. Ele pode ser chamado diretamente pela classe ou por uma instância da classe, mas não tem acesso aos atributos ou métodos da classe.

In [ ]:
class MinhaClasse:
    def __init__(self, valor):
        self.valor = valor

    @staticmethod
    def metodo_estatico(parametro):
        print(f'Este é um método estático e você passou o parâmetro {parametro}')

# Criando uma instância da classe
objeto = MinhaClasse(10)

# Acessando o atributo da instância
print(objeto.valor) # imprime 10

# Acessando o método estático
objeto.metodo_estatico(20) # imprime "Este é um método estático e você passou o parâmetro 20"
MinhaClasse.metodo_estatico(30) # imprime "Este é um método estático e você passou o parâmetro 30"

Existem várias funcionalidades do `@staticmethod`, mas basicamente ele é usado para definir um método que não precisa acessar nenhum atributo ou método da classe ou da instância. Este método é acessado diretamente pela classe e não precisa ser instanciado. Além disso, é possível acessar o método estático tanto através de uma instância da classe quanto diretamente pela classe.

Aqui temos como exemplo o uso do `@staticmethod` em um jogo de RPG :

In [ ]:
import random

class JogoRPG:
    def __init__(self, nome_jogador, classe_jogador):
        self.nome_jogador = nome_jogador
        self.classe_jogador = classe_jogador
        self.vida_jogador = 100

    def atacar(self, alvo):
        dano = 20
        alvo.vida_jogador -= dano
        print(f"{self.nome_jogador} atacou {alvo.nome_jogador} e causou {dano} de dano.")

    @staticmethod
    def gerar_inimigo():
        inimigos = ["Orc", "Lobo", "Goblin"]
        inimigo_gerado = inimigos[random.randint(0, len(inimigos)-1)]
        vida_inimigo = 50
        return inimigo_gerado, vida_inimigo

jogador1 = JogoRPG("Jogador 1", "Guerreiro")
jogador2 = JogoRPG("Jogador 2", "Mago")

inimigo, vida_inimigo = JogoRPG.gerar_inimigo()

print(f"Um inimigo {inimigo} foi gerado com {vida_inimigo} de vida.")

jogador1.atacar(jogador2)

Note que a função `gerar_inimigo` não precisa ser acessada via `self` e é chamada diretamente através da classe `JogoRPG`.

## 8. Métodos de Classe

O `@classmethod` é um decorador em Python que indica que o método seguinte é um método de classe. Isso significa que o método é chamado em relação à classe e não em relação a uma instância da classe.

Aqui está um exemplo de como usar `@classmethod` :

In [ ]:
class MinhaClasse:
    x = [1, 2, 3]

    @classmethod
    def modifica_x(cls, novo_valor):
        cls.x.append(novo_valor)

MinhaClasse.modifica_x(4)
print(MinhaClasse.x) # [1, 2, 3, 4]

Neste exemplo, `modifica_x` é um método de classe e pode ser chamado diretamente a partir da classe, sem precisar criar uma instância. O primeiro argumento do método é geralmente chamado de `cls`, que se refere à própria classe e permite que o método acesse e modifique variáveis ​​de classe.

É importante notar que, embora os métodos de classe sejam chamados diretamente na classe, eles ainda podem ser chamados a partir de uma instância da classe.

In [ ]:
obj = MinhaClasse()
obj.modifica_x(5)
print(obj.x) # [1, 2, 3, 4, 5]
print(MinhaClasse.x) # [1, 2, 3, 4, 5]

Os métodos de classe são úteis quando você deseja fornecer um método que não precisa acessar o estado da instância, mas precisa acessar o estado da classe.

O `@classmethod` é um decorador em Python que indica que o método seguinte é um método de classe. Os métodos de classe são diferentes dos métodos normais (ou métodos de instância) porque são chamados em relação à classe e não em relação a uma instância da classe. Isso significa que eles podem acessar e modificar variáveis ​​de classe, mas não podem acessar o estado de uma instância específica.

Um exemplo comum de uso de `@classmethod` é a criação de um método de fábrica para a criação de novas instâncias da classe. Por exemplo :

In [ ]:
class MinhaClasse:
    def __init__(self, valor):
        self.valor = valor

    @classmethod
    def cria_novo(cls, valor):
        return cls(valor)

obj = MinhaClasse.cria_novo(5)
print(obj.valor) # 5

Aqui, o método de fábrica `cria_novo` é chamado diretamente na classe, mas cria e retorna uma nova instância da classe com o valor especificado.

Outro exemplo é quando precisamos de um método que precisa acessar informações da classe, mas não precisa do estado de uma instância específica.

In [ ]:
class MinhaClasse:
    x = [1, 2, 3]

    @classmethod
    def print_x(cls):
        print(cls.x)

MinhaClasse.print_x() # [1, 2, 3]

É importante notar que os métodos de classe podem ser chamados tanto na classe quanto em uma instância da classe, mas o primeiro argumento do método sempre será a classe em si, e não a instância.

Além disso, os métodos de classe podem ser usados ​​com herança. Se uma subclasse herda de uma classe com um método de classe, o método de classe na subclasse irá acessar as variáveis ​​de classe da superclasse.

Em resumo, os métodos de classe são úteis quando você deseja fornecer um método que não precisa acessar o estado de uma instância específica, mas precisa acessar o estado da classe.

Segue um exemplo de uma calculadora :

In [ ]:
class Calculadora:
    def __init__(self, valor1, valor2):
        self.valor1 = valor1
        self.valor2 = valor2
    
    @classmethod
    def soma(cls, valor1, valor2):
        return valor1 + valor2

    @classmethod
    def subtracao(cls, valor1, valor2):
        return valor1 - valor2

    @classmethod
    def multiplicacao(cls, valor1, valor2):
        return valor1 * valor2

    @classmethod
    def divisao(cls, valor1, valor2):
        return valor1 / valor2

calculadora = Calculadora(10, 2)
print(calculadora.soma()) # 12
print(calculadora.subtracao()) # 8
print(calculadora.multiplicacao()) # 20
print(calculadora.divisao()) # 5.0

Note que os valores são passados como argumentos e não precisam ser acessados via `self`.

Confira mais um exemplo abaixo :

In [ ]:
class MinhaClasse:
    x = [1, 2, 3]

    @classmethod
    def modifica_x(cls, novo_valor):
        cls.x.append(novo_valor)

# Criação de uma instância da classe
obj = MinhaClasse()

# Imprimindo o valor de x antes da modificação
print(obj.x) # [1, 2, 3]

# Utilizando o método modifica_x como método de classe
MinhaClasse.modifica_x(4)

# Imprimindo o valor de x depois da modificação
print(obj.x) # [1, 2, 3, 4]

# Acessando o valor de x através da instância
print(obj.x) # [1, 2, 3, 4]

No exemplo acima, o método `modifica_x` é um método de classe que adiciona um novo valor à lista `x` da classe. Ele é chamado diretamente pela classe e modifica o valor de `x` para todas as instâncias da classe.

Aqui está mais um exemplo que demonstra as funcionalidades do @classmethod :

In [ ]:
class MinhaClasse:
    def __init__(self, valor):
        self.value = valor

    @classmethod
    def metodo_classe(cls):
        print("Este é um método de classe.")

    @classmethod
    def de_string(cls, string):
        valor = int(string)
        return cls(valor)

    def metodo_instancia(self):
        print("Este é um método de instância.")

# Criando uma instância de MinhaClasse
obj = MinhaClasse(10)

# Chamando um método de instância
obj.metodo_instancia() # Saída : Este é um método de instância.

# Chamando um método de classe
obj.metodo_classe() # Saída : Este é um método de classe.
MinhaClasse.metodo_classe() # Saída : Este é um método de classe.

# Usando o método de classe para criar uma instância
new_obj = MinhaClasse.de_string("20")
print(new_obj.value) # Saída : 20

Existem algumas coisas a serem observadas neste exemplo:

- O decorador `@classmethod` é usado para definir um método de classe;
- Os métodos de classe são chamados da mesma maneira que os métodos de instância, mas eles podem ser chamados diretamente na classe, sem precisar criar uma instância;
- Os métodos de classe recebem um parâmetro adicional chamado `cls`, que é uma referência à própria classe. Isso permite que os métodos de classe acessem e modifiquem os atributos e métodos da classe;
- O método `de_string` é um exemplo de como os métodos de classe podem ser usados para criar novas instâncias da classe, usando dados externos;

Aqui temos um exemplo de como os métodos de classe podem ser usados em um cenário de RPG :

In [ ]:
class Jogador:
    def __init__(self, nome, vida, ataque):
        self.nome = nome
        self.vida = vida
        self.ataque = ataque

    @classmethod
    def cria_guerreiro(cls):
        return cls("Guerreiro", 100, 20)

    @classmethod
    def cria_mago(cls):
        return cls("Mago", 75, 30)

    def leva_dano(self, dano):
        self.vida -= dano

    def ataca_inimigo(self, inimigo):
        inimigo.leva_dano(self.ataque)

# Criando um guerreiro e um mago usando métodos de classe
guerreiro = Jogador.cria_guerreiro()
mago = Jogador.cria_mago()

# Atacando um inimigo
guerreiro.ataca_inimigo(mago)
print(f'{mago.vida = }') # Saída : mago.vida = 55

Neste exemplo, a classe `Jogador` tem três atributos: `nome`, `vida`, e `ataque`. Ela também tem dois métodos de classe, `cria_guerreiro` e `cria_mago`, que são usados para criar novas instâncias da classe com valores pré-definidos. Isso permite que o jogador crie personagens prontos para jogar sem precisar inserir manualmente todas as informações. Os métodos de instância `leva_dano` e `ataca_inimigo` são usados para simular combate no jogo.

## 9. Métodos Mágicos

Os métodos mágicos do Python são uma espécie de ´atalhos´ de sintaxe que permitem que você customize a forma como seus objetos são tratados pelo interpretador Python. Esses métodos são identificados por nomes que começam e terminam com dois sublinhados, como `__init__` ou `__str__`.

Por exemplo, o método `__init__` é usado para inicializar um objeto quando ele é criado. Ele é chamado automaticamente quando você cria uma nova instância de uma classe.

O método `__str__` é usado para definir como um objeto deve ser representado como uma string. Ele é chamado quando você usa a função built-in `__str()__` para converter um objeto para uma string.

Além disso, existem outros métodos mágicos, como `__add__` para personalizar como a operação de adição deve ser realizada entre objetos, `__len__` para personalizar como a função built-in `__len()__` deve funcionar para um objeto específico, entre outros.

Os métodos mágicos do Python incluem :

- `__init__` : é usado para inicializar um objeto quando ele é criado. É chamado automaticamente quando você cria uma nova instância de uma classe;

In [ ]:
class MinhaClasse:
    def __init__(self, valor):
        self.valor = valor

objeto = MinhaClasse(5)
print(objeto.valor) # imprime 5

- `__str__` : é usado para definir como um objeto deve ser representado como uma string. Ele é chamado quando você usa a função built-in `str()` para converter um objeto para uma string;

In [ ]:
class MinhaClasse:
    def __init__(self, valor):
        self.valor = valor
        
    def __str__(self):
        return 'Valor: ' + str(self.valor)

objeto = MinhaClasse(5)
print(str(objeto)) # imprime "Valor: 5"

- `__repr__` : é usado para definir como um objeto deve ser representado como uma string de representação. Ele é chamado quando você usa a função built-in `repr()` para converter um objeto para uma string;

In [ ]:
class MinhaClasse:
    def __init__(self, valor):
        self.valor = valor
        
    def __repr__(self):
        return 'MinhaClasse({})'.format(self.valor)

objeto = MinhaClasse(5)
print(repr(objeto)) # imprime MinhaClasse(5)

- `__add__` : é usado para personalizar a operação de adição entre objetos. Ele é chamado quando você usa o operador `+` para adicionar dois objetos;

In [ ]:
class MinhaClasse:
    def __init__(self, valor):
        self.valor = valor
        
    def __add__(self, outro_objeto):
        return MinhaClasse(self.valor + outro_objeto.valor)

objeto1 = MinhaClasse(5)
objeto2 = MinhaClasse(10)
resultado = objeto1 + objeto2
print(resultado.valor) # imprime 15

- `__sub__` : é usado para personalizar a operação de subtração entre objetos. Ele é chamado quando você usa o operador `-` para subtrair dois objetos;

In [ ]:
class MinhaClasse:
    def __init__(self, valor):
        self.valor = valor
        
    def __sub__(self, outro_objeto):
        return MinhaClasse(self.valor - outro_objeto.valor)

objeto1 = MinhaClasse(5)
objeto2 = MinhaClasse(10)
resultado = objeto1 - objeto2
print(resultado.valor) # imprime -5

- `__mul__` : é usado para personalizar a operação de multiplicação entre objetos. Ele é chamado quando você usa o operador `*` para multiplicar dois objetos;

In [ ]:
class MinhaClasse:
    def __init__(self, valor):
        self.valor = valor
        
    def __mul__(self, outro_objeto):
        return MinhaClasse(self.valor * outro_objeto.valor)

objeto1 = MinhaClasse(5)
objeto2 = MinhaClasse(10)
resultado = objeto1 * objeto2
print(resultado.valor) # imprime 50

- `__truediv__` : é usado para personalizar a operação de divisão entre objetos. Ele é chamado quando você usa o operador `/` para dividir dois objetos;

In [ ]:
class MinhaClasse:
    def __init__(self, valor):
        self.valor = valor
        
    def __truediv__(self, outro_objeto):
        return MinhaClasse(self.valor / outro_objeto.valor)

objeto1 = MinhaClasse(10)
objeto2 = MinhaClasse(5)
resultado = objeto1 / objeto2
print(resultado.valor) # imprime 2.0

- `__floordiv__` : é usado para personalizar a operação de divisão inteira entre objetos. Ele é chamado quando você usa o operador `//` para dividir dois objetos;

In [ ]:
class MinhaClasse:
    def __init__(self, valor):
        self.valor = valor
        
    def __floordiv__(self, outro_objeto):
        return MinhaClasse(self.valor // outro_objeto.valor)

objeto1 = MinhaClasse(10)
objeto2 = MinhaClasse(5)
resultado = objeto1 // objeto2
print(resultado.valor) # imprime 2

- `__mod__` : é usado para personalizar a operação de módulo entre objetos. Ele é chamado quando você usa o operador `%` para obter o resto da divisão entre dois objetos;

In [ ]:
class MinhaClasse:
    def __init__(self, valor):
        self.valor = valor
        
    def __mod__(self, outro_objeto):
        return MinhaClasse(self.valor % outro_objeto.valor)

objeto1 = MinhaClasse(10)
objeto2 = MinhaClasse(5)
resultado = objeto1 % objeto2
print(resultado.valor) # imprime 0

- `__pow__` : é usado para personalizar a operação de exponenciação entre objetos. Ele é chamado quando você usa o operador `**` para elevar um objeto a uma potência;

In [ ]:
class MinhaClasse:
    def __init__(self, valor):
        self.valor = valor
        
    def __pow__(self, outro_objeto):
        return MinhaClasse(self.valor ** outro_objeto.valor)

objeto1 = MinhaClasse(2)
objeto2 = MinhaClasse(3)
resultado = objeto1 ** objeto2
print(resultado.valor) # imprime 8

- `__len__` : é usado para personalizar a função built-in `len()` para um objeto específico. Ele é chamado quando você usa a função `len()` para obter o comprimento de um objeto;

In [ ]:
class MinhaClasse:
    def __init__(self, valores):
        self.valores = valores
        
    def __len__(self):
        return len(self.valores)

objeto = MinhaClasse([1,2,3,4,5])
print(len(objeto)) # imprime 5

- `__getitem__` : é usado para personalizar o acesso a itens em um objeto. Ele é chamado quando você usa o operador `[]` para acessar um item em um objeto;

In [ ]:
class MinhaClasse:
    def __init__(self, valores):
        self.valores = valores
        
    def __getitem__(self, indice):
        return self.valores[indice]

objeto = MinhaClasse([1,2,3])
print(objeto[1]) # imprime 2

- `__setitem__` : é usado para personalizar a atribuição de valores a itens em um objeto. Ele é chamado quando você usa o operador `[]` para atribuir um item em um objeto;

In [ ]:
class MinhaClasse:
    def __init__(self, valores):
        self.valores = valores
        
    def __setitem__(self, indice, valor):
        self.valores[indice] = valor

objeto = MinhaClasse([1,2,3])
objeto[1] = 4
print(objeto.valores) # imprime [1, 4, 3]

- `__delitem__` : é usado para personalizar a remoção de itens em um objeto. Ele é chamado quando você usa a instrução `del` para remover um item de um objeto;

In [ ]:
class MinhaClasse:
    def __init__(self, valores):
        self.valores = valores
        
    def __delitem__(self, indice):
        del self.valores[indice]

objeto = MinhaClasse([1,2,3])
del objeto[1]
print(objeto.valores) # imprime [1, 3]

- `__iter__` : é usado para personalizar a iteração sobre um objeto. Ele é chamado quando você usa o operador `in` ou a instrução `for` para iterar sobre um objeto;

In [ ]:
class MinhaClasse:
    def __init__(self, valores):
        self.valores = valores
        self.indice = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.indice >= len(self.valores):
            raise StopIteration
        result = self.valores[self.indice]
        self.indice += 1
        return result

objeto = MinhaClasse([1,2,3])
for valor in objeto:
    print(valor)
# imprime 1
# imprime 2
# imprime 3

- `__next__` : é usado para personalizar a obtenção do próximo item em uma iteração. Ele é chamado quando você usa a função `next()` para obter o próximo item de um objeto;

In [ ]:
class MinhaClasse:
    def __init__(self, valores):
        self.valores = valores
        self.indice = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.indice >= len(self.valores):
            raise StopIteration
        result = self.valores[self.indice]
        self.indice += 1
        return result

objeto = MinhaClasse([1,2,3])
iterador = iter(objeto)
print(next(iterador)) # imprime 1
print(next(iterador)) # imprime 2
print(next(iterador)) # imprime 3

- `__call__` : é usado para personalizar a chamada de um objeto como uma função. Ele é chamado quando você usa parenteses `()` para chamar um objeto como se fosse uma função;

In [ ]:
class MinhaClasse:
    def __init__(self, valor):
        self.valor = valor
        
    def __call__(self, novo_valor):
        self.valor = novo_valor

objeto = MinhaClasse(5)
objeto(10)
print(objeto.valor) # imprime 10

- `__getattr__` : é usado para personalizar o acesso a atributos de um objeto. Ele é chamado quando você tenta acessar um atributo que não existe em um objeto;

In [ ]:
class MinhaClasse:
    def __init__(self, valor):
        self.valor = valor
        
    def __getattr__(self, name):
        if name == 'valor_dobrado':
            return self.valor * 2
        else:
            raise AttributeError

objeto = MinhaClasse(5)
print(objeto.valor_dobrado) # imprime 10

- `__setattr__` : é usado para personalizar a atribuição de valores a atributos de um objeto. Ele é chamado quando você usa o operador `=` para atribuir um valor a um atributo de um objeto;

In [ ]:
class MinhaClasse:
    def __init__(self):
        self.valor = None
        
    def __setattr__(self, name, value):
        if name == 'valor':
            value = value * 2
        super().__setattr__(name, value)

objeto = MinhaClasse()
objeto.valor = 5
print(objeto.valor) # imprime 10

- `__delattr__` : é usado para personalizar a remoção de atributos de um objeto. Ele é chamado quando você usa a instrução `del` para remover um atributo de um objeto;

In [ ]:
class MinhaClasse:
    def __init__(self, valor):
        self.valor = valor
        
    def __delattr__(self, name):
        if name == 'valor':
            print('Não é possível deletar o atributo valor')
        else:
            super().__delattr__(name)

objeto = MinhaClasse(5)
del objeto.valor # imprime "Não é possível deletar o atributo valor"

- `__eq__` : é usado para personalizar a comparação de igualdade entre objetos. Ele é chamado quando você usa o operador `==` para comparar dois objetos;

In [ ]:
class MinhaClasse:
    def __init__(self, valor):
        self.valor = valor
        
    def __eq__(self, outro_objeto):
        return self.valor == outro_objeto.valor

objeto1 = MinhaClasse(5)
objeto2 = MinhaClasse(5)
resultado = objeto1 == objeto2
print(resultado) # imprime True

- `__lt__` : é usado para personalizar a comparação `menor que` entre objetos. Ele é chamado quando você usa o operador `<` para comparar dois objetos;

In [ ]:
class MinhaClasse:
    def __init__(self, valor):
        self.valor = valor
        
    def __lt__(self, outro_objeto):
        return self.valor < outro_objeto.valor

objeto1 = MinhaClasse(5)
objeto2 = MinhaClasse(10)
resultado = objeto1 < objeto2
print(resultado) # imprime True

- `__le__` : é usado para personalizar a comparação `menor ou igual` entre objetos. Ele é chamado quando você usa o operador `<=` para comparar dois objetos;

In [ ]:
class MinhaClasse:
    def __init__(self, valor):
        self.valor = valor
        
    def __le__(self, outro_objeto):
        return self.valor <= outro_objeto.valor

objeto1 = MinhaClasse(5)
objeto2 = MinhaClasse(10)
resultado = objeto1 <= objeto2
print(resultado) # imprime True

- `__gt__` : é usado para personalizar a comparação `maior que` entre objetos. Ele é chamado quando você usa o operador `>` para comparar dois objetos;

In [ ]:
class MinhaClasse:
    def __init__(self, valor):
        self.valor = valor
        
    def __gt__(self, outro_objeto):
        return self.valor > outro_objeto.valor

objeto1 = MinhaClasse(10)
objeto2 = MinhaClasse(5)
resultado = objeto1 > objeto2
print(resultado) # imprime True

- `__ge__` : é usado para personalizar a comparação `maior ou igual` entre objetos. Ele é chamado quando você usa o operador `>=` para comparar dois objetos;

In [ ]:
class MinhaClasse:
    def __init__(self, valor):
        self.valor = valor
        
    def __ge__(self, outro_objeto):
        return self.valor >= outro_objeto.valor

objeto1 = MinhaClasse(10)
objeto2 = MinhaClasse(5)
resultado = objeto1 >= objeto2
print(resultado) # imprime True

Esses são alguns dos métodos mágicos mais comuns do Python, mas há outros. É importante ter em mente que esses métodos geralmente não devem ser chamados diretamente, mas sim sobrescritos e usados pelo interpretador Python automaticamente em determinadas situações.

Além disso, existem também métodos mágicos que são específicos para classes de tipos específicos, como métodos mágicos para classes de números inteiros, métodos mágicos para classes de sequências, métodos mágicos para classes de contêineres, entre outros.

É importante notar que o uso excessivo de métodos mágicos pode tornar seu código mais difícil de entender e manter, então é recomendado usá-los com moderação e apenas quando necessário.

## 10. Polimorfismo

`Polimorfismo` é uma característica da programação orientada a objetos que permite que objetos de diferentes classes sejam tratados como objetos de uma classe base comum. Isso significa que uma chamada de método pode ser feita em um objeto sem saber sua classe específica.

Exemplo 1:

In [ ]:
class Animal:
    def __init__(self, nome):
        self.nome = nome

    def falar(self):
        pass

class Cachorro(Animal):
    def falar(self):
        return "Au au"

class Gato(Animal):
    def falar(self):
        return "Miau"

animais = [Cachorro("Bobby"), Gato("Tom")]

for animal in animais:
    print(animal.falar())

Neste exemplo, a classe `Animal` é a classe base comum e as classes `Cachorro` e `Gato` são classes derivadas. Ambos os objetos `Cachorro` e `Gato` herdam o método `falar()` da classe base `Animal`, mas cada um tem uma implementação diferente. Quando o loop é executado, o método `falar()` é chamado em cada objeto sem saber sua classe específica, mas a implementação correta é chamada de acordo com a classe do objeto.

O polimorfismo permite que os objetos de diferentes classes sejam tratados de maneira semelhante, o que aumenta a reutilização de código e facilita a manutenção do código. Isso é especialmente útil quando você tem uma lista de objetos de diferentes classes e deseja iterar sobre eles, chamando um método comum, sem precisar saber a classe específica de cada objeto.

Exemplo 2:

In [ ]:
class Forma:
    def area(self):
        pass

class Quadrado(Forma):
    def __init__(self, lado):
        self.lado = lado
    def area(self):
        return self.lado * self.lado

class Circulo(Forma):
    def __init__(self, raio):
        self.raio = raio
    def area(self):
        return 3.14 * (self.raio ** 2)

formas = [Quadrado(10), Circulo(5)]

for forma in formas:
    print(forma.area())

Neste exemplo, a classe `Forma` é a classe base comum e as classes `Quadrado` e `Circulo` são classes derivadas. Ambos os objetos `Quadrado` e `Circulo` herdam o método `area()` da classe base `Forma`, mas cada um tem uma implementação diferente. Quando o loop é executado, o método `area()` é chamado em cada objeto sem saber sua classe específica, mas a implementação correta é chamada de acordo com a classe do objeto.

Exemplo 3:

In [ ]:
class Veiculo:
    def __init__(self, marca):
        self.marca = marca

    def andar(self):
        pass

class Carro(Veiculo):
    def andar(self):
        return "O carro da marca {} está andando".format(self.marca)

class Moto(Veiculo):
    def andar(self):
        return "A moto da marca {} está andando".format(self.marca)

veiculos = [Carro("Ford"), Moto("Yamaha")]

for veiculo in veiculos:
    print(veiculo.andar())

Neste exemplo, a classe `Veiculo` é a classe base comum e as classes `Carro` e `Moto` são classes derivadas. Ambos os objetos `Carro` e `Moto` herdam o método `andar()` da classe base `Veiculo`, mas cada um tem uma implementação diferente. Quando o loop é executado, o método `andar()` é chamado em cada objeto sem saber sua classe específica, mas a implementação correta é chamada de acordo com a classe do objeto.

Esses exemplos ilustram como o polimorfismo permite que os objetos de diferentes classes sejam tratados de maneira semelhante, o que aumenta a reutilização de código e facilita a manutenção do código.

Mais alguns exemplos usando o RPG :

In [ ]:
class Personagem:
    def __init__(self, nome):
        self.nome = nome
        self.vida = 100
        self.mana = 50

    def atacar(self):
        pass

class Guerreiro(Personagem):
    def atacar(self):
        return f"{self.nome} ataca com sua espada."

class Mago(Personagem):
    def atacar(self):
        return f"{self.nome} lança um feitiço."

personagens = [Guerreiro("Conan"), Mago("Gandalf")]

for personagem in personagens:
    print(personagem.atacar())

Neste exemplo, a classe `Personagem` é a classe base comum e as classes `Guerreiro` e `Mago` são classes derivadas. Ambos os objetos `Guerreiro` e `Mago` herdam o método `atacar()` da classe base `Personagem`, mas cada um tem uma implementação diferente. Quando o loop é executado, o método `atacar()` é chamado em cada objeto sem saber sua classe específica, mas a implementação correta é chamada de acordo com a classe do objeto.

Outro exemplo :

In [ ]:
class Arma:
    def __init__(self, nome, dano):
        self.nome = nome
        self.dano = dano

    def atacar(self):
        pass

class Espada(Arma):
    def atacar(self):
        return f"Ataque com {self.nome} causando {self.dano} de dano."

class Arco(Arma):
    def atacar(self):
        return f"Ataque com {self.nome} causando {self.dano} de dano."

armas = [Espada("Excalibur", 50), Arco("Longbow", 40)]

for arma in armas:
    print(arma.atacar())

Neste exemplo, a classe `Arma` é a classe base comum e as classes `Espada` e `Arco` são classes derivadas. Ambos os objetos `Espada` e `Arco` herdam o método `atacar()` da classe base `Arma`, mas cada um tem uma implementação diferente. Quando o loop é executado, o método `atacar()` é chamado em cada objeto sem saber sua classe específica, mas a implementação correta é chamada de acordo com a classe do objeto.

Esses exemplos ilustram como o polimorfismo pode ser usado para criar jogos RPG mais complexos, onde personagens e armas possuem habilidades e características diferentes. O polimorfismo permite que esses objetos sejam tratados de maneira semelhante, permitindo que o jogador interaja com eles de forma genérica, sem precisar saber sua classe específica. Isso também facilita a implementação de novos personagens e armas no jogo, pois eles podem herdar as características e métodos básicos da classe base e adicionar suas próprias características e métodos específicos.

## 11. Sobrecarga

A `sobrecarga` de operadores é uma técnica em Python que permite aos desenvolvedores definir como os operadores (+, -, *, etc.) funcionam com diferentes tipos de dados. Isso é feito através de métodos especiais chamados `magic methods` ou `dunder methods` (dunder é uma abreviação de `double underscore`).

Por exemplo, imagine que você tem uma classe chamada `Vetor2D` que representa um vetor com uma coordenada `x` e uma coordenada `y`. Você pode querer sobrecarregar o operador `+` para somar dois vetores juntos. Isso é feito através do método mágico `__add__`. Aqui está um exemplo de como fazer isso:

In [ ]:
class Vetor2D:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __add__(self, outro_vetor):
        x = self.x + outro_vetor.x
        y = self.y + outro_vetor.y
        return Vetor2D(x, y)

v1 = Vetor2D(1, 2)
v2 = Vetor2D(3, 4)
v3 = v1 + v2
print(v3.x)  # imprime 4
print(v3.y)  # imprime 6

No exemplo acima, quando você escreve `v1 + v2`, o Python automaticamente chama o método `__add__` na classe `Vetor2D`, passando `v1` como o primeiro argumento e `v2` como o segundo. Dentro do método `__add__`, você pode definir como os dois vetores devem ser somados juntos. No caso acima, estamos simplesmente adicionando as coordenadas `x` e `y` de cada vetor.

Além de `__add__`, existem outros métodos mágicos que você pode sobrecarregar para diferentes operadores, incluindo:

- `__sub__` para o operador `-`;
- `__mul__` para o operador `*`;
- `__truediv__` para o operador `/`;
- `__eq__` para o operador `==`;
- `__lt__` para o operador `<`;
- `__len__` para a função built-in `len()`;

É importante notar que a sobrecarga de operadores não é apenas para tipos de dados personalizados, mas também pode ser usada para tipos de dados built-in do python como `list`, `dict`, e etc.

Aqui estão alguns exemplos adicionais de sobrecarga de operadores em Python :

- Sobrecarga do operador `*` para multiplicação escalar :

In [ ]:
class Vetor2D:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __mul__(self, escalar):
        x = self.x * escalar
        y = self.y * escalar
        return Vetor2D(x, y)

v = Vetor2D(1, 2)
v_escalado = v * 3
print(v_escalado.x)  # imprime 3
print(v_escalado.y)  # imprime 6

- Sobrecarga do operador `==` para comparação de igualdade :

In [ ]:
class Pessoa:
    def __init__(self, nome, idade):
        self.nome = nome
        self.idade = idade

    def __eq__(self, outra_pessoa):
        return self.nome == outra_pessoa.nome and self.idade == outra_pessoa.idade

p1 = Pessoa("John", 30)
p2 = Pessoa("John", 30)
p3 = Pessoa("Jane", 25)
print(p1 == p2)  # imprime True
print(p1 == p3)  # imprime False

- Sobrecarga da função `len()` para retornar o tamanho de uma lista personalizada :

In [ ]:
class MinhaLista:
    def __init__(self, itens):
        self.itens = itens

    def __len__(self):
        return len(self.itens)

ml = MinhaLista([1, 2, 3, 4, 5])
print(len(ml))  # imprime 5

Aqui está um exemplo de como você poderia criar uma classe `Calculadora` que inverta as operações matemáticas usando sobrecarga de operadores :

In [ ]:
class Calculadora:
    def __init__(self, valor):
        self.valor = valor

    def __add__(self, outro_valor):
        return Calculadora(outro_valor - self.valor)

    def __sub__(self, outro_valor):
        return Calculadora(outro_valor + self.valor)

    def __mul__(self, outro_valor):
        return Calculadora(outro_valor / self.valor)

    def __truediv__(self, outro_valor):
        return Calculadora(outro_valor * self.valor)

    def __repr__(self):
        return str(self.valor)

calc = Calculadora(10)
resultado = calc + 20
print(resultado) # imprime -10
resultado = calc - 20
print(resultado) # imprime 30
resultado = calc * 2
print(resultado) # imprime 5
resultado = calc / 2
print(resultado) # imprime 20

Neste exemplo, a classe `Calculadora` tem um atributo `valor` que armazena o valor atual da calculadora. Os métodos `__add__`, `__sub__`, `__mul__` e `__truediv__` sobrecarregam os operadores de soma, subtração, multiplicação e divisão, respectivamente, para invertê-los. Por exemplo, o método `__add__` subtrai o valor da outra calculadora do valor atual em vez de adicioná-lo. No método `__repr__` é retornado o valor atual da calculadora.

Desta forma, quando você escreve `calc + 20`, o Python chama o método `__add__` na classe `Calculadora`, passando 20 como o segundo operando e retorna -10.

Aqui está um exemplo de como você poderia usar a sobrecarga de operadores para criar uma classe `Personagem` no RPG :

In [ ]:
class Personagem:
    def __init__(self, nome, vida, dano):
        self.nome = nome
        self.vida = vida
        self.dano = dano

    def __str__(self):
        return f'{self.nome} tem {self.vida} de vida e causa {self.dano} de dano'

    def __add__(self, outro_personagem):
        return Personagem(f'Grupo de {self.nome} e {outro_personagem.nome}',
                            self.vida + outro_personagem.vida,
                            self.dano + outro_personagem.dano)

    def __sub__(self, outro_personagem):
        return Personagem(f'{self.nome} sem {outro_personagem.nome}',
                            self.vida - outro_personagem.vida,
                            self.dano - outro_personagem.dano)

    def __mul__(self, outro_personagem):
        return Personagem(f'{self.nome} x {outro_personagem.nome}',
                            self.vida * outro_personagem.vida,
                            self.dano * outro_personagem.dano)

guerreiro = Personagem('Guerreiro', 100, 20)
mago = Personagem('Mago', 80, 30)
print(guerreiro) # imprime 'Guerreiro tem 100 de vida e causa 20 de dano'
print(mago) # imprime 'Mago tem 80 de vida e causa 30 de dano'
grupo = guerreiro + mago
print(grupo) # imprime 'Grupo de Guerreiro e Mago tem 180 de vida e causa 50 de dano'
duelo = guerreiro * mago
print(duelo) # imprime 'Guerreiro x Mago tem 8000 de vida e causa 600 de dano'

Neste exemplo, a classe `Personagem` tem três atributos: `nome`, `vida` e `dano`. O método `__str__` é usado para fornecer uma representação legível do objeto quando impresso. Os métodos `__add__`, `__sub__` e `__mul__` são usados para sobrecarregar os operadores `+`, `-` e `*`, respectivamente, para combinar dois personagens em um grupo, subtrair um personagem de outro ou multiplicar dois personagens. Cada operação retorna um novo objeto `Personagem` com atributos combinados ou modificados.

## 12. Iteradores

Você já deve ter notado que pode usar laços [for](https://docs.python.org/pt-br/3/reference/compound_stmts.html#for) com a maioria das coleções em Python :

In [ ]:
for elemento in [1, 2, 3]:
    print(elemento)
for elemento in (1, 2, 3):
    print(elemento)
for chave in {'one':1, 'two':2}:
    print(chave)
for caractere in "123":
    print(caractere)
for linha in open("aneis_poder.txt"):
    print(linha, end='')

Esse estilo de acesso é claro, conciso e conveniente. O uso de iteradores permeia e unifica o Python. Nos bastidores, a instrução [for](https://docs.python.org/pt-br/3/reference/compound_stmts.html#for) chama [iter()](https://docs.python.org/pt-br/3/library/functions.html#iter) no objeto contêiner. A função retorna um objeto iterador que define o método [__next__()](https://docs.python.org/pt-br/3/library/stdtypes.html#iterator.__next__) que acessa elementos no contêiner, um de cada vez. Quando não há mais elementos, [__next__()](https://docs.python.org/pt-br/3/library/stdtypes.html#iterator.__next__) levanta uma exceção [StopIteration](https://docs.python.org/pt-br/3/library/exceptions.html#StopIteration) que informa ao [for](https://docs.python.org/pt-br/3/reference/compound_stmts.html#for) para terminar. Você pode chamar o método [__next__()](https://docs.python.org/pt-br/3/library/stdtypes.html#iterator.__next__) usando a função embutida [next()](https://docs.python.org/pt-br/3/library/functions.html#next); este exemplo mostra como tudo funciona :

In [ ]:
letras = 'abc'
iterador = iter(letras)
print(iterador)
print(next(iterador))
print(next(iterador))
print(next(iterador))
print(next(iterador))
# StopIteration                             Traceback (most recent call last)
# Cell In[15], line 7
#       5 print(next(iterador))
#       6 print(next(iterador))
# ----> 7 print(next(iterador))

# StopIteration: 

Observando o mecanismo por trás do protocolo dos iteradores, fica fácil adicionar esse comportamento às suas classes. Defina um método `__iter__()` que retorna um objeto que tenha um método [__next__()](https://docs.python.org/pt-br/3/library/stdtypes.html#iterator.__next__). Se uma classe já define `__next__()`, então `__iter__()` pode simplesmente retornar `self`:

In [ ]:
class Reverte:
    """Iterador para repetir sobre uma sequência de trás para frente."""
    def __init__(self, dado):
        self.dado = dado
        self.indice = len(dado)

    def __iter__(self):
        return self

    def __next__(self):
        if self.indice == 0:
            raise StopIteration
        self.indice = self.indice - 1
        return self.dado[self.indice]

rev = Reverte('spam')
iter(rev)

for char in rev:
    print(char)

## 13. Geradores

[Geradores](https://docs.python.org/pt-br/3/glossary.html#term-generator) são uma ferramenta simples e poderosa para criar iteradores. São escritos como funções normais mas usam a instrução [yield](https://docs.python.org/pt-br/3/reference/simple_stmts.html#yield) quando precisam retornar dados. Cada vez que [next()](https://docs.python.org/pt-br/3/library/functions.html#next) é chamado, o gerador volta ao ponto onde parou (lembrando todos os valores de dados e qual instrução foi executada pela última vez). Um exemplo mostra como geradores podem ser trivialmente fáceis de criar:

In [ ]:
def reverte(dado):
    for indice in range(len(dado)-1, -1, -1):
        yield dado[indice]

for char in reverte('Supercalifragilisticexpialidocious'):
    print(char, end=' ')

Qualquer coisa que possa ser feita com geradores também pode ser feita com iteradores baseados numa classe, como descrito na seção anterior. O que torna geradores tão compactos é que os métodos `__iter__()` e `__next__()` são criados automaticamente.

Outro ponto chave é que as variáveis locais e o estado da execução são preservados automaticamente entre as chamadas. Isto torna a função mais fácil de escrever e muito mais clara do que uma implementação usando variáveis de instância como `self.index` e `self.data`.

Além disso, quando geradores terminam, eles levantam [StopIteration](https://docs.python.org/pt-br/3/library/exceptions.html#StopIteration) automaticamente. Combinados, todos estes aspectos tornam a criação de iteradores tão fácil quanto escrever uma função normal.

## 14. Expressões Geradoras

Alguns geradores simples podem ser codificados, de forma sucinta, como expressões, usando uma sintaxe semelhante a compreensões de lista, mas com parênteses em vez de colchetes. Essas expressões são projetadas para situações em que o gerador é usado imediatamente, pela função que o engloba. As expressões geradoras são mais compactas, mas menos versáteis do que as definições completas do gerador, e tendem a usar menos memória do que as compreensões de lista equivalentes.

In [ ]:
print(sum(i*i for i in range(10)))                 # soma dos quadrados

x_vetor = [10, 20, 30]
y_vetor = [7, 5, 3]
print(sum(x*y for x,y in zip(x_vetor, y_vetor)))

palavras_unicas = set(palavra for linha in open("aneis_poder.txt") for palavra in linha.split())
print(palavras_unicas)

orador = max((estudante.gpa, estudante.nome) for estudante in graduandos)

palavra = 'Supercalifragilisticexpialidocious'
print(list(palavra[i] for i in range(len(palavra)-1, -1, -1)))

## 15. Extra

### 12.1. Estados Dentro da Classe

Mantendo estados dentro da classe.

In [ ]:
class Cachorro():
    def __init__(self, nome, latindo=False):
        self.nome = nome
        self.latindo = latindo
    
    def latir(self):
        if self.latindo:
            print(f'O {self.nome} ainda está latindo!')
            return
        
        print(f'O {self.nome} está latindo...')
        self.latindo = True
    
    def parar_latir(self):
        if not self.latindo:
            print(f'O {self.nome} não está latindo!')
            return
        
        print(f'O {self.nome} está parando de latir...')
        self.latindo = False
    
    def roncar(self):
        if self.latindo:
            print(f'O {self.nome} não pode roncar enquanto late!')
            return
        
        print(f'O {self.nome} está roncando...')

fido = Cachorro('Fido')
rex = Cachorro('Rex')

fido.latir()
fido.latir()
fido.roncar()
fido.parar_latir()
fido.roncar()

### 12.2. \_\_dict__ e vars()

São maneiras de ver os atributos e valores de uma instância.

In [ ]:
class Pessoa():
    ano_atual = 2023

    def __init__(self, nome, idade):
        self.nome = nome
        self.idade = idade

    def get_ano_nasc(self):
        return Pessoa.ano_atual - self.idade

p1 = Pessoa('João', 65)
print(p1.__dict__)
print(vars(p1))

# ele pode ser editável, mas não é comum
p1.__dict__['novo'] = 'atributo'
print(p1.__dict__)
print(vars(p1))

# ou apagável
del p1.__dict__['idade']
print(p1.__dict__)
print(vars(p1))

Também é possível passar os dados de um dicionário para o objeto e criar (ou recriar) uma instância com eles.

In [ ]:
class Pessoa():
    ano_atual = 2023

    def __init__(self, nome, idade):
        self.nome = nome
        self.idade = idade

    def get_ano_nasc(self):
        return Pessoa.ano_atual - self.idade

dados = {'nome': 'João', 'idade': 65}
p1 = Pessoa(**dados)
print(p1.nome)
print(p1.idade)
